In [ ]:
#função para calcular a distancia de mahalanobis
def mahalanobis_distance(data_numeric):

    #Calculando a matrix de covariancia com np.cov.
    cov = np.cov(data_numeric, rowvar=False) #rowvar (opcional): É um parâmetro booleano que indica se cada linha de X representa uma variável (valor padrão True) ou se cada coluna representa uma variável (valor False).
'''
A matriz de covariância é uma medida estatística que captura as relações entre as variáveis em um conjunto de dados numéricos.
Ela fornece informações sobre como as diferentes variáveis se relacionam entre si e como elas variam conjuntamente.
'''
    #Verificando se a matrix de covariancia é simetrica para evitar problemas na inversão.
    if not np.allclose(cov, cov.T) or np.isnan(cov).any(): #usando transpose para comparar a simetria e verificando se existe valores nulos.
        raise ValueError('A matriz de covariância não é positiva definida ou contém NaNs')
'''
A verificação de simetria e ausência de valores NaN é realizada para garantir que a matriz de covariância seja válida e não apresente problemas antes de realizar a inversão.
'''
    #Calculando a matriz de covariância inversa
    inv_cov = np.linalg.inv(cov)
'''
A função np.linalg.inv é uma função da biblioteca NumPy (numpy) que calcula a matriz inversa de uma matriz dada.
Neste caso, a função np.linalg.inv é usada para calcular a matriz inversa da matriz de covariância cov, resultando na matriz de covariância inversa inv_cov.
'''
    #Calculando a media de cada variavel
    vars_mean = data_numeric.mean(axis=0).values
'''
A linha de código acima calcula a média de cada variável no conjunto de dados data_numeric e armazena os valores médios em um array chamado vars_mean.
Essa média será usada posteriormente no cálculo da distância de Mahalanobis para cada linha do conjunto de dados.
'''
    # Calculando a distancia Mahalanobis para cada linha de data_numeric
    diff = data_numeric - vars_mean #subtrai a média de cada variável
    md = np.sqrt(np.einsum('ij,ij->i', diff.dot(inv_cov), diff))

'''
diff.dot(inv_cov): Essa operação realiza o produto matricial entre a matriz diff e a matriz de covariância inversa inv_cov.
O resultado é uma matriz resultante do produto das diferenças pela matriz de covariância inversa. Essa etapa é um cálculo intermediário necessário para o cálculo da distância de Mahalanobis.

np.einsum('ij,ij->i', diff.dot(inv_cov), diff): Essa operação utiliza a função np.einsum para calcular o produto interno entre cada linha da matriz resultante do passo anterior (diff.dot(inv_cov))
 e a própria linha, resultando em um array unidimensional. Esse cálculo é equivalente ao somatório do produto dos elementos de cada linha da matriz pela linha correspondente, mas de forma mais eficiente.

np.sqrt(...): Finalmente, a função np.sqrt é aplicada ao resultado do passo anterior para calcular a raiz quadrada de cada elemento do array.
Isso resulta na distância de Mahalanobis para cada linha do conjunto de dados data_numeric.
'''

    #Calculando a media e o desvio padrão da distancia de Mahalanobis
    md_mean = md.mean()
    md_std = md.std()

    return md, md_mean, md_std

#função para remover outliers do conjunto de treinamento
def remove_outliers(data, threshold=3):
    #Filtrando apenas as colunas numericas
    numeric_cols = data.select_dtypes(include=[np.number]).columns
    data_numeric = data[numeric_cols]

    # Calculando a distancia de Mahalanobis do conjunto de treino
    md, md_mean, md_std = mahalanobis_distance(data_numeric)

    #Calcula o threshold para remoção dos outliers
    threshold = md_mean + threshold * md_std
'''
Cálculo do limite para remoção de outliers:
Com base na média e no desvio padrão da distância de Mahalanobis, é calculado um limite que define quais valores serão considerados outliers.
O limite é definido multiplicando o desvio padrão por um valor de threshold fornecido como parâmetro. Quanto maior o valor do threshold, mais tolerante será a remoção de outliers.
'''
    #Identificando os outliers
    outliers = data[md > threshold]
'''
Identificação dos outliers:
Os outliers são identificados selecionando as linhas do conjunto de dados cuja distância de Mahalanobis seja maior que o limite calculado.
Essas linhas representam os pontos que se afastam significativamente do padrão dos dados.
'''
    #Remove os outliers
    data = data[md <= threshold].values
'''
Remoção dos outliers:
Os outliers são removidos do conjunto de dados, mantendo apenas as linhas cuja distância de Mahalanobis seja menor ou igual ao limite calculado.
'''
    return data

#função para remover outliers do conjunto de teste com base no threshold do conjunto de treinamento.
def remove_outliers_test(train_data, test_data, threshold=3):

    #Filtrando apenas colunas numericas do conjunto de treino
    numeric_cols = train_data.select_dtypes(include=[np.number]).columns
    train_numeric = train_data[numeric_cols]

    # calculando a distancia de Mahalanobis para train_numeric
    md_train, md_train_mean, md_train_std = mahalanobis_distance(train_numeric)

    #Calculando o limite para outliers(threshold) no conjunto de teste com base nas estatísticas do conjunto de treino
    threshold_test = md_train_mean + threshold * md_train_std

    #Filtrando apenas as colunas numericas no conjunto de teste
    test_numeric = test_data[numeric_cols]

    #Calculando a distância de Mahalanobis para test_numeric
    md_test, _, _ = mahalanobis_distance(test_numeric)

    #Identificando outliers no conjunto de teste com base nas estatisticas do conjunto de treinamento
    outliers = test_data[md_test > threshold_test]

    # Removendo Outliers do conjunto de teste
    test_data = test_data[md_test <= threshold_test].values

    return test_data